# Amazon Bedrock에서 Claude 3.5 Sonnet 모델을 사용해 주어진 토픽에 대한 파워포인트 슬라이드를 만들어보기

### 환경 설정

#### 참조:  Bedrock 호출
- 참고: https://docs.anthropic.com/en/api/claude-on-amazon-bedrock


In [17]:
# pip install boto3

In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1: Prompt Chaining을 통한 대용량 파워포인트 슬라이드 생성

- 목표: 최소한의 API 호출로 Chaining Prompt를 사용하여 20장의 파워포인트 슬라이드를 생성하기
- 방법: 20장 분량의 고품질 프레젠테이션 내용을 생성하기 위해 Prompt Chaining 기법을 적용합니다. 이 과정은 다음과 같이 3단계로 구성됩니다.
    1. 전체 프레젠테이션 개요 작성
    2. 개요에 기반한 1-10번 슬라이드 내용 생성
    3. 개요에 기반한 11-20번 슬라이드 내용 생성
    각 단계의 결과를 다음 단계의 입력으로 사용하여 전체 프레젠테이션의 일관성을 유지합니다.


### 템플릿 파일 기술

In [19]:
from util.cross_region_inference_prompt_util import *

# 템플릿 파일의 경로를 지정
prompt_folder = "prompt_template"
outline_prompt_file_name = "outline_prompt_template.txt"

### Outline prompt template 로딩 및 토픽 추가

In [20]:
from jinja2 import Template

prompt_file_path = os.path.join(prompt_folder, outline_prompt_file_name)

template_content = load_template_from_file(prompt_file_path)
PROMPT_TEMPLATE = Template(template_content)

TOPIC = "Paris 2024 Olympics Closing Ceremony Success Prediction"
outline_prompt = PROMPT_TEMPLATE.render(topic=TOPIC)

print(outline_prompt)

You are an expert in creating impactful and professional PowerPoint presentations. 
Create a detailed outline for a 20-slide presentation in Korean, ensuring a logical flow and comprehensive coverage of the topic.

Analyze the given topic thoroughly to write in the provided <output_format> JSON format.
Return only the JSON object without additional tags or explanations.

<context>
Paris 2024 Olympics Closing Ceremony Success Prediction
</context>

<instruction>
1. Title:
   - Create a compelling and concise main title that captures the essence of the presentation.

2. Slides:
   - Generate 20 slides, including an opening and a closing slide.
   - For each slide:
     - Assign a slide_number (01 to 20).
     - Create a clear and informative slide_title.
     
3. Content Guidelines:
   - Ensure a logical flow and structure throughout the presentation.
   - Include an introduction slide (slide 02) and a conclusion slide (slide 19).   

4. Common Points:
   - Include the most important key

### 전체 프레젠테이션의 Outline을 만드는 prompt 실행

In [21]:
print("\n## Generating Overall Outline by Claude 3.5 Sonnet:\n")
outline, input_tokens, output_tokens = invoke_claude(outline_prompt)

print(outline)

print("\n" + "="*50 + "\n")
print(f"Input tokens: {input_tokens}, Output tokens: {output_tokens}")
print("\n" + "="*50 + "\n")


## Generating Overall Outline by Claude 3.5 Sonnet:

{
    "title": "2024 파리 올림픽 폐회식 성공 예측: 글로벌 축제의 대미",
    "slides": [
        {
            "slide_number": "01",
            "slide_title": "2024 파리 올림픽 폐회식 성공 예측"
        },
        {
            "slide_number": "02",
            "slide_title": "소개: 올림픽 폐회식의 중요성"
        },
        {
            "slide_number": "03",
            "slide_title": "파리 2024 올림픽: 개요 및 특징"
        },
        {
            "slide_number": "04",
            "slide_title": "과거 올림픽 폐회식 분석: 성공 요인"
        },
        {
            "slide_number": "05",
            "slide_title": "파리의 문화적 자산: 폐회식 활용 방안"
        },
        {
            "slide_number": "06",
            "slide_title": "기술 혁신: 폐회식에서의 첨단 기술 적용"
        },
        {
            "slide_number": "07",
            "slide_title": "지속가능성: 친환경 폐회식 계획"
        },
        {
            "slide_number": "08",
            "slide_title": "글로벌 참여: 온라인 플랫폼 활용 전략"
        },
        {
            "slide_number": "0

In [22]:
# Outline에서 슬라이드 타이틀만 보기
print_slide_titles(outline)

프레젠테이션 제목: 2024 파리 올림픽 폐회식 성공 예측: 글로벌 축제의 대미

슬라이드 01: 2024 파리 올림픽 폐회식 성공 예측
슬라이드 02: 소개: 올림픽 폐회식의 중요성
슬라이드 03: 파리 2024 올림픽: 개요 및 특징
슬라이드 04: 과거 올림픽 폐회식 분석: 성공 요인
슬라이드 05: 파리의 문화적 자산: 폐회식 활용 방안
슬라이드 06: 기술 혁신: 폐회식에서의 첨단 기술 적용
슬라이드 07: 지속가능성: 친환경 폐회식 계획
슬라이드 08: 글로벌 참여: 온라인 플랫폼 활용 전략
슬라이드 09: 안전 및 보안: 대규모 행사 관리 방안
슬라이드 10: 문화적 다양성: 국제 협력 및 교류 증진
슬라이드 11: 예술 공연: 프랑스 예술의 정수 선보이기
슬라이드 12: 미디어 전략: 글로벌 중계 및 홍보 계획
슬라이드 13: 관객 경험: 현장 및 원격 참여 최적화
슬라이드 14: 레거시 계획: 폐회식 이후 영향력 유지
슬라이드 15: 경제적 영향: 폐회식의 경제 파급 효과
슬라이드 16: 디지털 혁신: AR/VR 기술의 활용
슬라이드 17: 국제 협력: IOC 및 각국 NOC와의 협업
슬라이드 18: 위험 관리: 잠재적 문제 및 대응 전략
슬라이드 19: 결론: 성공적인 폐회식을 위한 핵심 요소
슬라이드 20: 질의응답 및 토론


### Slide prompt template 로딩

In [23]:
# Slide Prompt template
prompt_file_name = "slide_prompt_template.txt"

slide_prompt_file_path = os.path.join(prompt_folder, prompt_file_name)

template_content = load_template_from_file(slide_prompt_file_path)
slide_prompt_template = Template(template_content)
slide_prompt = slide_prompt_template.render()

print("\n## Slide prompt template...")
print(slide_prompt)



## Slide prompt template...
You are an expert in creating impactful and professional PowerPoint presentations.
Your task is to create detailed content for a specific slide in 20-slide presentation in Korean, ensuring a logical flow and comprehensive coverage of the topic.

Based on this  of the , Generate detailed content for slides  to .
For each slide, include:
1. Slide title
2. Main content points (3-5 bullet points)
3. Relevant data or examples
4. Suggesion for visuals (e.g., image, chart, graph)
5. A brief speaker note (1-2 sentences)


### 만들어진 슬라이드를 저장할 결과 폴더 생성

In [24]:
from datetime import datetime
import os

currentDateAndTime = datetime.now()
currentTime = currentDateAndTime.strftime("%Y-%m-%d-%H-%M-%S")
# print("The current time is", currentTime)

output_directory = "./results"
output_directory = os.path.join(output_directory, currentTime)
os.makedirs(output_directory, exist_ok=True)

### Slide prompt template을 기반으로 슬라이드 1 - 10 번을 생성할 Slide prompt 생성

In [25]:
from util.cross_region_inference_prompt_util import create_slide_prompt, generate_slide_content
# topic = "Paris 2024 Olympics Closing Ceremony Success Prediction"

print("\n## Generating prompt for creating the content of slides 1-10...")
slide_prompt = create_slide_prompt(slide_prompt_template, TOPIC, outline, 1, 10)
print(slide_prompt)

# 1 - 10번째 슬라이드 생성
print("\n## Generating content for slides 1-10...")
include_outline = False
slides_1_10 = generate_slide_content(slide_prompt, outline, output_directory, include_outline, 1, 10, 1)


## Generating prompt for creating the content of slides 1-10...
You are an expert in creating impactful and professional PowerPoint presentations.
Your task is to create detailed content for a specific slide in 20-slide presentation in Korean, ensuring a logical flow and comprehensive coverage of the topic.

Based on this {
    "title": "2024 파리 올림픽 폐회식 성공 예측: 글로벌 축제의 대미",
    "slides": [
        {
            "slide_number": "01",
            "slide_title": "2024 파리 올림픽 폐회식 성공 예측"
        },
        {
            "slide_number": "02",
            "slide_title": "소개: 올림픽 폐회식의 중요성"
        },
        {
            "slide_number": "03",
            "slide_title": "파리 2024 올림픽: 개요 및 특징"
        },
        {
            "slide_number": "04",
            "slide_title": "과거 올림픽 폐회식 분석: 성공 요인"
        },
        {
            "slide_number": "05",
            "slide_title": "파리의 문화적 자산: 폐회식 활용 방안"
        },
        {
            "slide_number": "06",
            "slide_title": "기술 혁신: 폐회식에서

### Slide prompt template을 기반으로 슬라이드 11 - 20 번을 생성할 Slide prompt 생성

In [26]:
from util.cross_region_inference_prompt_util import create_slide_prompt, generate_slide_content
# topic = "Paris 2024 Olympics Closing Ceremony Success Prediction"

print("\nGenerating prompt for creating the content of slides 11-20...")
slide_prompt = create_slide_prompt(slide_prompt_template, TOPIC, outline, 11, 20)
print(slide_prompt)

# 11 - 20번째 슬라이드 생성
print("\nGenerating content for slides 11-20...")
include_outline = False
slides_1_10 = generate_slide_content(slide_prompt, outline, output_directory, include_outline, 11, 20, 2)


Generating prompt for creating the content of slides 11-20...
You are an expert in creating impactful and professional PowerPoint presentations.
Your task is to create detailed content for a specific slide in 20-slide presentation in Korean, ensuring a logical flow and comprehensive coverage of the topic.

Based on this {
    "title": "2024 파리 올림픽 폐회식 성공 예측: 글로벌 축제의 대미",
    "slides": [
        {
            "slide_number": "01",
            "slide_title": "2024 파리 올림픽 폐회식 성공 예측"
        },
        {
            "slide_number": "02",
            "slide_title": "소개: 올림픽 폐회식의 중요성"
        },
        {
            "slide_number": "03",
            "slide_title": "파리 2024 올림픽: 개요 및 특징"
        },
        {
            "slide_number": "04",
            "slide_title": "과거 올림픽 폐회식 분석: 성공 요인"
        },
        {
            "slide_number": "05",
            "slide_title": "파리의 문화적 자산: 폐회식 활용 방안"
        },
        {
            "slide_number": "06",
            "slide_title": "기술 혁신: 폐회식에서의 

### 두 차례 호출로 만들어졌던 슬라이드 내용을 한 파일에 최종적으로 모으기

In [27]:
# Merge results
merge_files(output_directory)

All files have been merged into 'final_result.txt' in the directory './results/2024-08-29-04-28-06'.


## 2: Zero-shot prompting 기반의 고급 프롬프트 기법 활용
* Role-based prompting: 특정 역할을 가정한 프롬프트 생성
* Task-specific instructions: 구체적인 작업 지침 제공
* Content structuring: 체계적인 콘텐츠 구조화
* Output constraints: 출력 형식 및 내용에 대한 제약 설정

### 프롬프트 로딩

In [28]:
from jinja2 import Template

all_prompt_file_name = "allinall_prompt_template.txt"

prompt_file_path = os.path.join(prompt_folder, all_prompt_file_name)

template_content = load_template_from_file(prompt_file_path)
PROMPT_TEMPLATE = Template(template_content)

TOPIC = "Paris 2024 Olympics Closing Ceremony Success Prediction"
presentation_prompt = PROMPT_TEMPLATE.render(topic=TOPIC)
# print_json(prompt)
print(presentation_prompt)

You are an expert in creating impactful and professional PowerPoint presentations. 
Create a detailed outline for a 20-slide presentation in Korean, 
ensuring a logical flow and comprehensive coverage of the topic.

<topic>
Paris 2024 Olympics Closing Ceremony Success Prediction
</topic>

<instruction>
For each slide, provide:
1. A clear and concise title
2. 3-5 bullet points of key information
3. Any relevant data or statistics (if applicable)

Structure the presentation as follows:
- Slide 1: Introduction
- Slides 2-3: Background and Context
- Slides 4-7: Main Points and Analysis
- Slides 8-9: Predictions and Potential Outcomes
- Slide 10: Conclusion and Key Takeaways

Present the content for all 20 slides in a clear, organized manner. 
The number of output tokens for each slide must not exceed 100
</instruction>


### 생성한 슬라이드 결과를 저장할 폴더 생성

In [29]:
from datetime import datetime
import os

currentDateAndTime = datetime.now()
currentTime = currentDateAndTime.strftime("%Y-%m-%d-%H-%M-%S")
# print("The current time is", currentTime)

output_directory = "./results"
output_directory = os.path.join(output_directory, currentTime)
os.makedirs(output_directory, exist_ok=True)

### 슬라이드 생성

In [30]:
from util.cross_region_inference_prompt_util import generate_all_slide_content

generate_all_slide_content(presentation_prompt, output_directory)

Generated Presentation Content:
1. 소개: 파리 2024 올림픽 폐회식 성공 예측
• 발표 주제 소개
• 폐회식의 중요성 강조
• 파리 2024 올림픽의 의의

2. 배경: 올림픽 폐회식의 역사
• 최초의 현대 올림픽 폐회식 (1896년)
• 주요 폐회식 이정표
• 폐회식의 진화와 발전
• 최근 폐회식 트렌드

3. 맥락: 파리의 올림픽 개최 경험
• 1900년 파리 올림픽
• 1924년 파리 올림픽
• 파리의 문화적 중요성
• 프랑스의 스포츠 전통

4. 주요 포인트: 폐회식 장소 - 샹젤리제
• 샹젤리제의 상징성
• 예상 관중 수: 약 100만 명
• 물류 및 보안 과제
• 도시 경관을 활용한 연출 가능성

5. 분석: 기술 혁신과 폐회식
• AR/VR 기술 활용 계획
• 드론 쇼의 잠재력
• 지속 가능한 기술 사용
• 글로벌 시청자 참여 방안

6. 주요 포인트: 문화적 요소 통합
• 프랑스 예술과 문화 쇼케이스
• 다양성과 포용성 강조
• 세계 문화와의 조화
• 올림픽 정신 구현 방안

7. 분석: 지속 가능성 및 환경 고려사항
• 친환경 재료 사용
• 탄소 중립 목표
• 재사용 및 재활용 계획
• 지속 가능한 교통 솔루션

8. 예측: 폐회식 주요 하이라이트
• 예상 공연 유형
• 유명 아티스트 참여 가능성
• 혁신적인 기술 쇼케이스
• 감동적인 폐회 선언 순간

9. 잠재적 결과: 경제적 영향
• 예상 관광객 수: 15백만 명
• 경제 효과 예측: €10억 이상
• 장기적 도시 이미지 개선
• 향후 국제 행사 유치 가능성

10. 결론 및 주요 시사점
• 파리 2024 폐회식의 성공 가능성
• 혁신과 전통의 조화 중요성
• 지속 가능성 강조의 의미
• 글로벌 화합의 메시지 전달

11. 폐회식 준비 현황
• 조직위원회 구성
• 주요 일정 및 마일스톤
• 예산 배정 현황
• 협력 파트너 및 스폰서

12. 안전 및 보안 대책
• 테러 대비 계획
• 군경 동원 규모: 약 45,000명
• 첨단 보안 기술 활용
• 관중 관리 전